## Rotation of the force vector

Analyse how the force vector rotates based on the rotation of a water molecule computed in minimal basis.

In [ ]:
import numpy as np
import plotly.express as px
from instance_mongodb import instance_mongodb_sei

from pymatgen.core.structure import Molecule

import torch

from monty.serialization import loadfn, dumpfn

from e3nn import o3

from utils import rotate_three_dimensions, subdiagonalize_matrix

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.minimal_basis

forces = []
angles = []

for doc in collection.find({'tags.group': 'rotated_water_molecules', 'orig.rem.basis': 'sto-3g', 'input.job_type': 'force'}):
    _force = doc['output']['gradients']
    forces.append(_force)
    angles.append(doc['tags']['angles'])

forces = np.array(forces)
angles = np.array(angles)

In [ ]:
# Plot the forces
fig = px.imshow(
    forces, animation_frame=0, labels=dict(x="Direction", y="Atom", color="Force (Ha/Bohr)"),
)
fig.update_xaxes(ticktext=['x', 'y', 'z'], tickvals=[0, 1, 2])
# Set integer ticks on the y-axis
fig.update_yaxes(tickmode='array', tickvals=[0, 1, 2])
fig.show()

In [ ]:
irreps_hamiltonian = o3.Irreps('1x1o')

D_matrices = []
for idx, angle in enumerate(angles):
    alpha, beta, gamma = angle

    rotation_matrix = rotate_three_dimensions(alpha, beta, gamma)
    rotation_matrix = torch.tensor(rotation_matrix)
    if idx == 0:
        rotation_matrix_0 = rotation_matrix

    # Reference the rotation matrix to the first one
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T

    D_matrix = irreps_hamiltonian.D_from_matrix(rotation_matrix)
    D_matrices.append(D_matrix)

D_matrices = torch.stack(D_matrices)
# Convert to numpy array
D_matrices = D_matrices.detach().numpy()

fig = px.imshow(
    D_matrices, animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)

fig.update_layout(title_text='Wigner Matrix (D)', title_x=0.5)
fig.show()

In [ ]:
forces_rotated = []
forces_computed = []
all_data = []

for i in range(len(angles)):
    forces_computed.append(forces[i, 0, :])

    force_rotated = forces[0, 0, :] @ D_matrices[i].T
    forces_rotated.append(force_rotated)

    difference = force_rotated - forces[i, 0, :]

    all_data.append(np.stack([force_rotated, forces[i, 0, :], difference]))

all_data = np.array(all_data)
print(all_data.shape)
# Plot with max and min values of -0.05 and 0.05
fig = px.imshow(
    all_data, animation_frame=0, labels=dict(x="Direction", y="Atom", color="Force (Ha/Bohr)"),
    range_color=[-0.05, 0.05]
)
fig.show()
